## Part 1: Extracting Company Names
**This part of the code will extract company names and save them to a CSV file.**

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [2]:
# Set up the WebDriver (Chrome in this case)
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Uncomment this line if you want to run the browser in headless mode
driver = webdriver.Chrome(options=options)

# Function to extract company names
def extract_company_names(driver):
    company_names = set()
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        companies = driver.find_elements(By.CLASS_NAME, "heading")
        for company in companies:
            try:
                name = company.text.strip()
                if name and name not in company_names:
                    company_names.add(name)
            except Exception as e:
                print(f"Error extracting company name: {e}")
        
        # Scroll down to load more companies
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(9)  # Adjust sleep time as needed to wait for the page to load
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    return list(company_names)

try:
    # Navigate to the EXHIBITOR LIST page
    driver.get("https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor")

    # Wait for the page to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

    # Extract company names
    all_company_names = extract_company_names(driver)

    # Save the company names to a CSV file
    df_names = pd.DataFrame(all_company_names, columns=["Company Name"])
    df_names.to_csv('company_names.csv', index=False)
    print("Company names saved to company_names.csv")

finally:
    # Close the WebDriver
    driver.quit()


Company names saved to company_names.csv


## Part 2: Extracting Detailed Information
**This part of the code will read the company names from the CSV file, normalize the names for URL generation, and extract detailed information from the profile pages.**

In [4]:
# Set up the WebDriver (Chrome in this case)
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Uncomment this line if you want to run the browser in headless mode
driver = webdriver.Chrome(options=options)

# Function to normalize company names for URL
def normalize_company_name(name):
    normalization_map = {
            "ALİENFENCE": "alenfence",
            "AṬLAS": "alas"
        }
    if name in normalization_map:
        return normalization_map[name]
    name = name.lower()
    name = name.replace(' ', '-')
    name = name.replace('&', '')
    name = name.replace('(', '')
    name = name.replace(')', '')
    name = name.replace('é', '')
    name = name.replace('ü', '')
    name = name.replace('ó', '')
    name = name.replace('ş', '')
    name = name.replace('è', '')
    name = name.replace('ü', '')
    name = name.replace('ö', '')
    name = name.replace('â', '')
    name = name.replace('\'', '')
    name = name.replace('/', '')
    name = name.replace(',', '')
    name = name.replace('.', '')
    name = name.replace('+', '')
    name = name.replace('~', '')
    name = name.replace(':', '')
    name = '-'.join(name.split())  # Remove multiple consecutive spaces and dashes
    return name

# Function to extract detailed information from the profile page
def extract_detailed_info(driver, url):
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h4.group.card-title.inner.list-group-item-heading'))
        )
        name = driver.find_element(By.CSS_SELECTOR, 'h4.group.card-title.inner.list-group-item-heading').text
        stand_info = driver.find_element(By.CSS_SELECTOR, 'div.head_discription p').text
        country = driver.find_element(By.CSS_SELECTOR, 'div.head_discription p span').text.strip()
        description = driver.find_element(By.CSS_SELECTOR, 'p.group.inner.float-left').text.strip()

        # Extract categories
        categories_elements = driver.find_elements(By.CSS_SELECTOR, 'ul.sector_block li')
        categories = [category.text for category in categories_elements]

        return {
            "Company Name": name,
            "Stand Info": stand_info,
            "Country": country,
            "Description": description,
            "Skills": ", ".join(categories)
        }
    except TimeoutException:
        print(f"Timeout loading page: {url}")
        return None
    except NoSuchElementException as e:
        print(f"Element not found on page {url}: {e}")
        return None
    except Exception as e:
        print(f"Error extracting detailed data for a company: {e}")
        return None

try:
    # Read company names from the CSV file
    df_names = pd.read_csv('company_names.csv')
    all_company_names = df_names['Company Name'].tolist()

    # Extract detailed information for each company
    all_companies_data = []
    base_url = "https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/"

    for company_name in all_company_names:
        normalized_name = normalize_company_name(company_name)
        company_url = base_url + normalized_name
        print(f"Scraping data for {company_name}: {company_url}")
        company_info = extract_detailed_info(driver, company_url)
        if company_info:
            all_companies_data.append(company_info)
        time.sleep(5)  # Adjust sleep time as needed to wait for the page to load

    # Save the data into a pandas DataFrame
    df = pd.DataFrame(all_companies_data)
    df.to_csv('exhibitors_data.csv', index=False)
    print("Data saved to exhibitors_data.csv")

finally:
    # Close the WebDriver
    driver.quit()


Scraping data for POLYTEL-EVOLIS: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/polytel-evolis
Scraping data for YAZ AGRI SOLUTIONS: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/yaz-agri-solutions
Scraping data for YAKEEY: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/yakeey
Scraping data for SWIFT BIZ SOLUTIONS PVT LTD/DIGIT: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/swift-biz-solutions-pvt-ltddigit
Scraping data for BOTGUARD OÜ: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/botguard-o
Scraping data for WEST COAST OPTILINKS LTD.: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/west-coast-optilinks-ltd
Scraping data for ORBIT HEALTH: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/orbit-he

Scraping data for HAKTRAK COMPANY FOR CYBERSECURITY: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/haktrak-company-for-cybersecurity
Scraping data for CDG: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/cdg
Scraping data for FEDERATION APEBI: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/federation-apebi
Scraping data for AHAD INTERNATIONAL FZC LIMITED- ICT ~ RECITAL FINANCE: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/ahad-international-fzc-limited--ict--recital-finance
Element not found on page https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/ahad-international-fzc-limited--ict--recital-finance: Message: no such element: Unable to locate element: {"method":"css selector","selector":"p.group.inner.float-left"}
  (Session info: chrome=124.0.6367.208); For documentation on this

Scraping data for HANGZHOU JIANYE CABLE CO.,LTD: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/hangzhou-jianye-cable-coltd
Scraping data for SPLASH GUY RECRUITMENT: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/splash-guy-recruitment
Scraping data for AZARA CONSULTING: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/azara-consulting
Scraping data for INGECYS TECHNOLOGIES: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/ingecys-technologies
Scraping data for DOCLINE: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/docline
Scraping data for LOPWORKS LIMITED: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/lopworks-limited
Scraping data for ACER: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/acer
Scra

Scraping data for UCOPIA BY WEBLIB: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/ucopia-by-weblib
Scraping data for CASABLANCA FINANCE CITY AUTHORITY: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/casablanca-finance-city-authority
Scraping data for DEALKHIR: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/dealkhir
Scraping data for AGIRH: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/agirh
Scraping data for DATALOGIC SRL: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/datalogic-srl
Scraping data for WATERFALL SECURITY SOLUTION: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/waterfall-security-solution
Scraping data for WIALON: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/wialon
Scraping data 

Scraping data for ALİENFENCE: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/alenfence
Scraping data for WIBIOCARD: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/wibiocard
Scraping data for ELM: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/elm
Scraping data for L'WORKSHOP: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/lworkshop
Scraping data for FOCUS MESSENGER: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/focus-messenger
Scraping data for KIWONTECH: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/kiwontech
Scraping data for NHAKRETXEU: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/nhakretxeu
Scraping data for CORAIL: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibi

...

Scraping data for STEAMLEDGE LIMITED: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/steamledge-limited
Scraping data for ONEFOR HOLDING GMBH: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/onefor-holding-gmbh
Scraping data for DRC SYSTEMS INDIA LIMITED: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/drc-systems-india-limited
Scraping data for GEOTECH: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/geotech
Scraping data for LAFRICAMOBILE: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/lafricamobile
Scraping data for SMART ENGINES SERVICE LLC: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/smart-engines-service-llc
Scraping data for AGRI DONE VISION: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/a

Timeout loading page: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/fujifilm-–-printing-solutions
Scraping data for INSIGHTEX: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/insightex
Scraping data for OMNISHORE GROUPE MEDTECH: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/omnishore-groupe-medtech
Scraping data for ARRAWTECH: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/arrawtech
Scraping data for SOFTWAREAG: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/softwareag
Element not found on page https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/softwareag: Message: no such element: Unable to locate element: {"method":"css selector","selector":"p.group.inner.float-left"}
  (Session info: chrome=124.0.6367.208); For documentation on this error

Scraping data for SCHOOLIFY: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/schoolify
Scraping data for BEAM TEKNOLOJI: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/beam-teknoloji
Scraping data for WISPER: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/wisper
Scraping data for DATAPROTECT: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/dataprotect
Scraping data for INFOBIP: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/infobip
Element not found on page https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/infobip: Message: no such element: Unable to locate element: {"method":"css selector","selector":"p.group.inner.float-left"}
  (Session info: chrome=124.0.6367.208); For documentation on this error, please visit: https://www.selenium.dev/doc

Scraping data for SECFINITY: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/secfinity
Scraping data for IEPM: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/iepm
Scraping data for GLOBAL ENTERTAINMENT: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/global-entertainment
Scraping data for DOJAH INC: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/dojah-inc
Scraping data for HUBX: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/hubx
Scraping data for GROUPEMENT DU PAIEMENT MOBILE MAROC: https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/groupement-du-paiement-mobile-maroc
Element not found on page https://exhibitors-dwtc.exhibitoronlinemanual.com/gitex-africa-2024/Exhibitor/ExbDetails/groupement-du-paiement-mobile-maroc: Message: no such element: 

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=124.0.6367.208)
Stacktrace:
	GetHandleVerifier [0x00007FF79AE31522+60802]
	(No symbol) [0x00007FF79ADAAC22]
	(No symbol) [0x00007FF79AC67CE4]
	(No symbol) [0x00007FF79AC605B3]
	(No symbol) [0x00007FF79AC513E4]
	(No symbol) [0x00007FF79AC52C14]
	(No symbol) [0x00007FF79AC51711]
	(No symbol) [0x00007FF79AC51049]
	(No symbol) [0x00007FF79AC50D42]
	(No symbol) [0x00007FF79AC4EBE4]
	(No symbol) [0x00007FF79AC4F22C]
	(No symbol) [0x00007FF79AC6A9F9]
	(No symbol) [0x00007FF79ACFAB7E]
	(No symbol) [0x00007FF79ACDAB7A]
	(No symbol) [0x00007FF79ACFA224]
	(No symbol) [0x00007FF79ACDA923]
	(No symbol) [0x00007FF79ACA8FEC]
	(No symbol) [0x00007FF79ACA9C21]
	GetHandleVerifier [0x00007FF79B1341BD+3217949]
	GetHandleVerifier [0x00007FF79B176157+3488183]
	GetHandleVerifier [0x00007FF79B16F0DF+3459391]
	GetHandleVerifier [0x00007FF79AEEB8E6+823622]
	(No symbol) [0x00007FF79ADB5FBF]
	(No symbol) [0x00007FF79ADB

In [ ]:
AṬLAS